In [42]:
from qiskit import QuantumCircuit, Aer, transpile
import numpy as np
from numpy import random
import matplotlib.pyplot as plt

backend = Aer.get_backend('qasm_simulator')

In [108]:
def shor_encoding(bit, addErrors=False, protectErrors=True):
    qc = QuantumCircuit(9,9)

    if(bit == 1):
        qc.x(0)

    qc.cx(0,3)
    qc.cx(0,6)

    qc.h(0)
    qc.h(3)
    qc.h(6)

    qc.cx(0,1)
    qc.cx(3,4)
    qc.cx(6,7)
    qc.cx(0,2)
    qc.cx(3,5)
    qc.cx(6,8)

    qc.barrier()

    # Random corruption of qubit 0
    # corruptedQubit = 0

    # Random corruption of any single qubit
    corruptedQubit = np.random.randint(0, 9)
    
    if(addErrors):
        error = np.random.choice([qc.x, qc.y, qc.z])
        error(corruptedQubit)

    # for qubit in qc.qubits:
    #    np.random.choice([qc.x, qc.y, qc.z])(qubit)
    
    qc.barrier()

    qc.cx(0,1)
    qc.cx(3,4)
    qc.cx(6,7)
    qc.cx(0,2)
    qc.cx(3,5)
    qc.cx(6,8)

    qc.ccx(1,2,0)
    qc.ccx(4,5,3)
    qc.ccx(7,8,6)

    qc.h(0)
    qc.h(3)
    qc.h(6)

    qc.cx(0,3)
    qc.cx(0,6)

    qc.ccx(3,6,0)

    qc.barrier()
    
    if(not protectErrors):
        error(corruptedQubit)
    
    qc.measure(0, 0)

    job = backend.run(transpile(qc, backend))
    return job.result().get_counts(qc)

In [109]:
def string_to_binary(string):
    binary = ''.join(format(ord(char), 'b') for char in string)
    return binary

def binary_to_string(binary):
    chunks = [binary[i:i+7] for i in range(0, len(binary), 7)]
    decimals = [int(chunk, 2) for chunk in chunks]
    string = ''.join(chr(decimal) for decimal in decimals)
    return string

In [117]:
def string_demo(input_string, protectErrors=True):
    print("Input String: " + input_string)
    
    input_bitstream = [int(bit) for bit in string_to_binary(input_string)]
    output_bitstream = ''
    for bit in input_bitstream:
        output = list(shor_encoding(bit, True, protectErrors).keys())[0]
        output_bitstream += output[len(output)-1]

    output_string = binary_to_string(output_bitstream)
    print("Output String: " + output_string)

In [118]:
string_demo("test")

Input String: test
Output String: test


In [119]:
string_demo("test", False)

Input String: test
Output String: tEs|
